
## はじめに  
本プログラムは前進型有限差分法による材料中の熱・水分の移動計算を行うものである。  
プログラムの基本的な考え方は以下に示すとおりである。

#### セルとは
- 差分法を用いるにあたり、壁体をいくつかのセルに分割する。  
- ここでセルとは複数の要素によって構成される集合体である。
- 主な構成要素は固体（材料骨格部・氷・塩結晶）、液体（水・溶液）、気体（空気・水蒸気）である。
- それぞれの構成要素は異なる物理量・状態量を保持している。
- 一方で、差分化されたセル内では局所平衡が成り立っているものと仮定し、3相の温度や気液水の化学ポテンシャルなどは平衡しているものとする。 
- 水分伝導率などの各種移動係数は、固体と液体、気体全ての物理量に影響を受けるため、各要素が保持するというよりはセルが有する情報とする。

#### 流量  
- 流れとは、隣り合う2つのセルが熱力学的非平衡状態になった際に生じるものである。  
- 基本的に熱や物質の流れは隣接するセルのうち、同じ要素の間で生じるものである。（隣接するセル間では液水と材料の間で熱伝導は生じない）
- 


- セルの質点の置き方について
    セルの分割方法、質点の分割の仕方についてはいくつか方法が考えられる。
    - 質点を全てセルの中心に置く方法（収支計算が簡単）　⇔　
    - 質点を全てセルの端に置く方法（流量計算が簡単）　⇔　異なる材料の水分収支を解く際に平衡含水率をどうするのかが難しい
    - 境界面のセルのみ質点を端に置き、他はすべてセルの中央に置く方法（質点間の距離計算が複雑）　⇒　計算結果はきれいだがプログラムが膨大に
    - 質点を全てセルの中心に置き、境界面のセルのみ体積を半分にする方法（体積の計算が複雑）

<img src="picture/readme_no1.jpg" width= 60%>  

### 本プログラムでできること

#### 計算内容（収支式）  
- 熱計算  
- 熱、水分計算（蒸気拡散支配領域）  
- 熱・水分計算（液水領域を含む）  

#### 流量計算方法  
- 熱（固体熱伝導）（水分の相変化熱）  
- 水蒸気（圧力基準　or 水分化学ポテンシャル基準）（絶対湿度基準も実装予定）  
- 液水（圧力基準　or 水分化学ポテンシャル基準）  

#### 境界条件  
- 第一種境界条件（温湿度固定）  
- 第二種境界条件（流量固定）  
- 第三種境界条件（セル間の流量計算）  


拡張性



## 各モジュール（ファイル）の説明  

各モジュールの内容および動かすのに必要なファイルは以下に示すとおりである。

#### 各モジュールを統括するファイル
- 1D_heat_moisture_main.ipynb  
    内容；各モジュールを用いることでセルのセッティング、流量計算、収支計算までの一連の流れを示す。  
    必要ファイル；ほぼ全て  
  
  
#### Cellを構成するモジュール群
- gas_water.ipynb  
    内容；気体（水蒸気）の状態量・物質量を保持するモジュール  
    必要ファイル；moisture_conversion.ipynb  


- liquid_water.ipynb  
    内容；液体（純水）の状態量・物理量を保持するモジュール  
    必要ファイル；property_conversion.ipynb  


- solid_material.ipynb  
    内容；固体（材料骨格部）の状態量・物理量を保持するモジュール  
    必要ファイル；property_conversion.ipynb  


#### 流量計算に関するモジュール
- flux.ipynb  
    内容；熱・水分の流量計算式を管理するモジュール  
    必要ファイル；なし
   

#### 収支計算に関するモジュール  
- balance_equation.ipynb  
    内容；熱・水分の収支計算式を管理するモジュール  
    必要ファイル；なし


#### 材料の物性値に関するモジュール群   
- property_conversion.ipynb  
    内容；各物性値ファイルから必要な情報を取得し、流量計算に適した形に変換するモジュール  
    必要ファイル；vapour_pressure, 各種物性値ファイル


- van_genuchten.ipynb  
    内容；mualeam - vanGenuchtenの水分特性曲線から不飽和透水係数を算出するモデル  
    必要ファイル；なし  


#### 水分の状態量・物質量の変換に関するモジュール群  
⇒gas_waterモジュールに加えても良い気がする

- vapour_pressure.ipynb  
    内容；温度と水分化学ポテンシャルから水蒸気圧・飽和水蒸気圧・水蒸気圧の温度微分を求めるモジュール  
    必要ファイル；なし
    

- moisture_conversion.ipynb
    内容；水蒸気の水分状態（相対湿度・絶対湿度・水蒸気圧・水分化学ポテンシャル）を変換するモジュール  
    必要ファイル；vapour_pressure.ipynb 



### 差分法による計算の基本手順  
本プログラムにおける基本的な計算手順は以下のようになる。  

<img src="caluculationMethod.jpg" width= 60%>  


#### 各クラスに従属するモジュール  
・材料物性モジュール(Cell)  
・セル間の差分を計算するモジュール(Flux)  
・流量計算の式を保管するモジュール(Flux)  
・第三種境界条件に用いる境界と隣接する空気の温湿度を管理するモジュール(BoundaryCondition)  
・第三種境界条件に用いる流量計算式を保管するモジュール(BoundaryCondition)  
・熱・水分の収支を保管するモジュール(BalanceEquation)  
・エラー判定をするモジュール(BalanceEquation)

### 計算・各クラスのフロー図

<img src="flow_diagram.jpg" width= 80%>  
